In [1]:
pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
poetry 1.6.1 requires jsonschema<4.18.0,>=4.10.0, but you have jsonschema 4.19.0 which is incompatible.
textbase-client 0.1.3 requires openai<0.28.0,>=0.27.8, but you have openai 1.30.1 which is incompatible.


In [8]:
import getpass
import os
os.environ["OPENAI_API_KEY"] = getpass.getpass()


 ········


ModuleNotFoundError: No module named 'langchain_community'

In [12]:

pip install jina "jina[torch]" jina-hub transformers


     ---------------------------------------- 0.0/374.6 kB ? eta -:--:--
     ---------- --------------------------- 102.4/374.6 kB 3.0 MB/s eta 0:00:01
     -------------------------------- ----- 317.4/374.6 kB 3.9 MB/s eta 0:00:01
     -------------------------------------  368.6/374.6 kB 2.9 MB/s eta 0:00:01
     -------------------------------------- 374.6/374.6 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement jina-hub (from versions: none)
ERROR: No matching distribution found for jina-hub


In [7]:
pip install langchain-community

   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   --- ------------------------------------ 0.2/2.0 MB 4.6 MB/s eta 0:00:01
   ------- -------------------------------- 0.4/2.0 MB 6.3 MB/s eta 0:00:01
   ------------ --------------------------- 0.6/2.0 MB 4.9 MB/s eta 0:00:01
   ---------------- ----------------------- 0.9/2.0 MB 4.9 MB/s eta 0:00:01
   --------------------- ------------------ 1.1/2.0 MB 4.9 MB/s eta 0:00:01
   ------------------------- -------------- 1.3/2.0 MB 4.9 MB/s eta 0:00:01
   ----------------------------- ---------- 1.5/2.0 MB 4.8 MB/s eta 0:00:01
   --------------------------------- ------ 1.7/2.0 MB 4.8 MB/s eta 0:00:01
   -------------------------------------- - 1.9/2.0 MB 4.7 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 4.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   -- ------------------------------------- 0.1/2.1 MB 3.6 MB/s eta 0:00:01
   ----- -----------------

In [18]:
pip install langchain

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ------ --------------------------------- 0.2/1.0 MB 10.9 MB/s eta 0:00:01
   ------------ --------------------------- 0.3/1.0 MB 6.9 MB/s eta 0:00:01
   ------------------------- -------------- 0.6/1.0 MB 5.7 MB/s eta 0:00:01
   --------------------------------- ------ 0.8/1.0 MB 5.4 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 4.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install langchain_chroma langchain_core langchain_text_splitters

     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ---------------------------------------- 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached typing_extensions-4.11.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Using cached asgiref-3.8.1-py3-none-any.whl.metadata (9.3 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/526.8 kB ? eta -:--:--
   ---------------- ----------------------- 215.0/526.8 kB 4.4 MB/s eta 0:00:01

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
poetry 1.6.1 requires build<0.11.0,>=0.10.0, but you have build 1.2.1 which is incompatible.
poetry 1.6.1 requires jsonschema<4.18.0,>=4.10.0, but you have jsonschema 4.19.0 which is incompatible.
textbase-client 0.1.3 requires openai<0.28.0,>=0.27.8, but you have openai 1.30.1 which is incompatible.


In [5]:
from langchain_openai import ChatOpenAI
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma.vectorstores import Chroma
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [6]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [7]:
rag_chain.invoke("What is Task Decomposition?")

NameError: name 'rag_chain' is not defined

In [ ]:
vectorstore.delete_collection()